In [2]:
import cv2
import numpy as np

In [3]:
# loading images
img1 = cv2.imread('./image_data/box1.png')
img2 = cv2.imread('./image_data/box2.png')

In [4]:
sift = cv2.SIFT_create() # initializing SIFT detector

In [5]:
# find the keypoints and descriptors
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

In [6]:
# find matches using k nearest neighbours
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

In [7]:
# apply ratio test to threshold the best matches
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

In [8]:
# draw matches
img3 = cv2.drawMatchesKnn(
    img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

In [9]:
# using cv2 for displaying the image
window_name = 'Feature Matching'
cv2.imshow(window_name, img3)

# waits for users to press any key
# (this is necessary to avoid Python kernel from crashing)
cv2.waitKey(0)

-1